# Sentiment Analysis of Market News

This project aims to analyse the news realated to a perticular stock and give out a general sentiment towards said stock. 

## Getting Libraries

In [277]:
import pandas as pd
import matplotlib
import numpy as np
import nltk
from pprint import pprint
from nltk.tokenize import word_tokenize
import spacy
import re
from nltk.sentiment import SentimentIntensityAnalyzer
#nltk.download()
import requests
from newscatcherapi import NewsCatcherApiClient
from bs4 import BeautifulSoup

## Getting User input and validating the input

In [279]:
ticker=input("Enter a ticker for stock ")

Enter a ticker for stock goog


In [280]:
ticker=ticker.lower()

In [102]:
#get stock and tickers for said stock

stock_name=["google","apple","microsoft"]
stock_ticker=["GOOG","AAPL","MSFT"]


## Getting information from News API

In [283]:
#using a request call to get news
url = ('https://newsapi.org/v2/everything?q='+ticker+'&apiKey=74cb9f5e7dc24fa2bfbf0f6056f72075')


response = requests.get(url)

#getting relavent information from JSON object
news=response.json()["articles"]
news_url=[]

##Getting the headlines from diffrent sources
for i in range(len(news)):
    news_url.append(news[i]['title'])
    


Testing a different API

In [109]:
newscatcherapi = NewsCatcherApiClient(x_api_key='KZYb52V3CtGrq9Yh6479s9DMFb5RZYlwZD6uqiZnCt0')

In [284]:
all_articles = newscatcherapi.get_search(q=ticker,
                                         lang='en',
                                         countries='CA',
                                         page_size=100)   

In [286]:
#getting relavent information from JSON object
news_stock=all_articles['articles']

stock_headline=[]
summary_stock=[]

In [287]:
#getting the summary for the news related to the stock
for i in range(len(news_stock)):
    summary_stock.append(news_stock[i]['summary'])
    

In [288]:
#getting the headlines for the news related to the stock
for i in range(len(news_stock)):
    stock_headline.append(news_stock[i]['title'])

In [289]:
stock_headline

['Teachers Retirement System of The State of Kentucky Takes $95.99 Million Position in Alphabet Inc. (NASDAQ:GOOG)',
 'AWM Capital LLC Has $815,000 Stake in Alphabet Inc. (NASDAQ:GOOG)',
 'Ambassador Advisors LLC Purchases 3,792 Shares of Alphabet Inc. (NASDAQ:GOOG)',
 'Industrial Alliance Investment Management Inc. Buys 267,185 Shares of Alphabet Inc. (NASDAQ:GOOG)',
 'Eagle Bay Advisors LLC Raises Stake in Alphabet Inc. (NASDAQ:GOOG)',
 'Legend Financial Advisors Inc. Makes New Investment in Alphabet Inc. (NASDAQ:GOOG)',
 'Spotlight Asset Group Inc. Buys Shares of 5,460 Alphabet Inc. (NASDAQ:GOOG)',
 'National Pension Service Boosts Stock Holdings in Alphabet Inc. (NASDAQ:GOOG)',
 'WealthPlan Investment Management LLC Acquires New Holdings in Alphabet Inc. (NASDAQ:GOOG)',
 'Continental Investors Services Inc. Makes New $10.86 Million Investment in Alphabet Inc. (NASDAQ:GOOG)',
 'Power Corp of Canada Grows Stake in Alphabet Inc. (NASDAQ:GOOG)',
 'Gould Capital LLC Purchases 301 Shares

## Analysing the News (Sentiment Analysis)

Pre-Processing

stock_headline (has the headlines from news catcher)
summary_stock (has the summary from news catcher)
news_url (has the headlines from news api)

The corpus had characters, extra whitespaces, 's, numbers, stopwords

In [291]:
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)

In [292]:
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

In [293]:
def remove_irr_char_func(text):
    '''
    Removes all irrelevant characters (numbers and punctuation) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without irrelevant characters
    '''
    return re.sub(r'[^a-zA-Z]', ' ', text)

In [213]:
##pre-processing 
##testing (not used)
tokenized_summary_raw = [word_tokenize(i) for i in summary_stock]
tokenized_summary_raw = [word_tokenize(i) for i in news_url]
tokenized_summary= [item for sublist in tokenized_summary_raw for item in sublist]

In [236]:
stopwords = nltk.corpus.stopwords.words("english")

In [237]:
#words = [w for w in tokenized_summary if w.lower() not in stopwords]

In [244]:
word=" ".join(tokenized_summary)

In [239]:
word=" ".join(words)

In [245]:
word=remove_irr_char_func(word)

In [246]:
word=remove_punctuation_func(word)

In [247]:
word=remove_extra_whitespaces_func(word)

Sentiment Analysis of Headlines

In [258]:
#initiating Pre-Trained Sentiment Analyzer 
sia = SentimentIntensityAnalyzer()

{'neg': 0.076, 'neu': 0.783, 'pos': 0.141, 'compound': 0.9982}

In [294]:
#testing 
sentiment=sia.polarity_scores(word)
sentiment

{'neg': 0.076, 'neu': 0.783, 'pos': 0.141, 'compound': 0.9982}

In [295]:
#getting a sentiment score for each headline
results = []

for line in stock_headline:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

In [296]:
#converting it to a dataframe
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.0,1.000,0.000,0.000,Teachers Retirement System of The State of Ken...
1,0.0,1.000,0.000,0.000,"AWM Capital LLC Has $815,000 Stake in Alphabet..."
2,0.0,0.804,0.196,0.296,"Ambassador Advisors LLC Purchases 3,792 Shares..."
3,0.0,0.833,0.167,0.296,Industrial Alliance Investment Management Inc....
4,0.0,1.000,0.000,0.000,Eagle Bay Advisors LLC Raises Stake in Alphabe...


In [297]:
#labeling headlines as positive or negative based on compound score.
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

,neg,neu,pos,compound,headline,label
0,0.0,1.000,0.000,0.000,Teachers Retirement System of The State of Ken...,0
1,0.0,1.000,0.000,0.000,"AWM Capital LLC Has $815,000 Stake in Alphabet...",0
2,0.0,0.804,0.196,0.296,"Ambassador Advisors LLC Purchases 3,792 Shares...",1
3,0.0,0.833,0.167,0.296,Industrial Alliance Investment Management Inc....,1
4,0.0,1.000,0.000,0.000,Eagle Bay Advisors LLC Raises Stake in Alphabe...,0


In [299]:
#Displaying the positive headlines
print("Positive headlines:\n")
pprint(list(df[df['label'] == 1].headline)[:5], width=200)



Positive headlines:

['Ambassador Advisors LLC Purchases 3,792 Shares of Alphabet Inc. (NASDAQ:GOOG)',
 'Industrial Alliance Investment Management Inc. Buys 267,185 Shares of Alphabet Inc. (NASDAQ:GOOG)',
 'Spotlight Asset Group Inc. Buys Shares of 5,460 Alphabet Inc. (NASDAQ:GOOG)',
 'National Pension Service Boosts Stock Holdings in Alphabet Inc. (NASDAQ:GOOG)',
 'Gould Capital LLC Purchases 301 Shares of Alphabet Inc. (NASDAQ:GOOG)']


In [301]:
#Displaying the negative headlines
print("\nNegative headlines:\n")
pprint(list(df[df['label'] == -1].headline)[:5], width=200)


Negative headlines:

[]


In [302]:
df

,neg,neu,pos,compound,headline,label
0,0.0,1.000,0.000,0.0000,Teachers Retirement System of The State of Ken...,0
1,0.0,1.000,0.000,0.0000,"AWM Capital LLC Has $815,000 Stake in Alphabet...",0
2,0.0,0.804,0.196,0.2960,"Ambassador Advisors LLC Purchases 3,792 Shares...",1
3,0.0,0.833,0.167,0.2960,Industrial Alliance Investment Management Inc....,1
4,0.0,1.000,0.000,0.0000,Eagle Bay Advisors LLC Raises Stake in Alphabe...,0
5,0.0,1.000,0.000,0.0000,Legend Financial Advisors Inc. Makes New Inves...,0
6,0.0,0.657,0.343,0.5719,"Spotlight Asset Group Inc. Buys Shares of 5,46...",1
7,0.0,0.796,0.204,0.3182,National Pension Service Boosts Stock Holdings...,1
8,0.0,1.000,0.000,0.0000,WealthPlan Investment Management LLC Acquires ...,0
9,0.0,1.000,0.000,0.0000,Continental Investors Services Inc. Makes New ...,0
